In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
customer_id = '137'
customer_name='%kimballwest%'
buckets = "Fixings & Fasteners"
# strategy_version_input=705
# attribute_id_input=5131
formatted_attribute = 'outside_diameter_sae'

In [3]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
field='OutsideDiameter'
fields = [field]

# df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df], axis = 1)
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']

4467


In [5]:
# print(len(df[(df['OutsideDiameter'].astype(str)!='nan')&(df['mm'].astype(str)!='[]')]))
# df[(df['OutsideDiameter'].astype(str)!='nan')&(df['mm'].astype(str)!='[]')]
df.head(0)

Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, OutsideDiameter]
Index: []

In [6]:
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

[]


In [7]:
# df['MaxPressure'].explode().value_counts()

In [12]:
pattern=r'''(?i)(\d*.?\d+\/\d+\s?")|()''' 
df['name'] = df['product_name'].apply(lambda x: re_extract(pattern, str(x)))
df['ld'] = df['long_desc'].apply(lambda x: re_extract(pattern, str(x)))



pat=r'(?i)(\d*.?\d+\/\d+)|()'
df['matches']=df[field].apply(lambda x: re_extract(pat, str(x)))

custom=df[(df[field].astype(str)!='nan')&((df['matches'].astype(str)!='[]'))]
custom[curation_col]=custom['matches'].apply(lambda x: re.sub(r'.+to\s?|','',str(x))).apply(lambda x: re.sub(r"'|\[|\]",'',str(x))).apply(lambda x: re.sub(r'(?i)\s?mm\s?',' mm',str(x))).apply(lambda x: f'{x}"').apply(lambda x: re.sub(r'""','"',str(x)))                                        
print(len(custom))
matchcustom=custom[['external_id',curation_col]]
# custom[curation_col].explode().value_counts()
# # custom[custom[curation_col].astype(str)==r'300 150   ']
matchcustom[0:500]

563


external_id Q:outside_diameter_sae
3634      511822               1-11/16"
3635      511826                 2-1/8"
3637      511713                   3/4"
3638      511729                 1-3/4"
3642      511712                 11/16"
3643      511930                 2-1/2"
3644      511711                   5/8"
3645      511931                 2-5/8"
3647       50953                  9/16"
3648      511716                 15/16"
3649      511828                 2-3/8"
3650      511811                1-1/16"
3651      511710                  9/16"
3652      511932                 2-3/4"
3653      511818                 1-1/2"
3654      501062                   5/8"
3655      511928                 2-1/4"
3656      511817                1-7/16"
3657      511714                 13/16"
3658      511715                   7/8"
3659      511815                1-5/16"
3660      511723                 1-3/8"
3661       50941                   3/4"
3663      511936                 3-1/4"
3664      511718                1-1/16"
3665      511927                 2-1/8"
3666      511830                 2-5/8"
3667      511819                1-9/16"
3668       50957                 13/16"
3669      511824                 1-7/8"
3670      511812                 1-1/8"
3671      511816                 1-3/8"
3673      511834                 3-1/8"
3674      511832                 2-7/8"
3675      501061                   5/8"
3676      511820                 1-5/8"
3677      511938                 3-1/2"
3678      511829                 2-1/2"
3679      511732               1-15/16"
3680      511814                 1-1/4"
3681      511728               1-11/16"
3682       50939                  9/16"
3684      511722                1-5/16"
3685      511720                1-3/16"
3686      511742                2-9/16"
3687      511827                 2-1/4"
3688      511929                 2-3/8"
3689       50963                1-1/16"
3690      511823                 1-3/4"
3691      511721                 1-1/4"
3692      511925                 1-7/8"
3693      511724                1-7/16"
3694      511738                2-5/16"
3695      511813                1-3/16"
3696       50952                   1/2"
3697      511725                 1-1/2"
3698      511831                 2-3/4"
3699      511719                 1-1/8"
3700      511726                1-9/16"
3701      511727                 1-5/8"
3702      511730               1-13/16"
3703      501060                  5/16"
3704      511734                2-1/16"
3705      511736                2-3/16"
3706      511740                2-7/16"
3707      511744               2-11/16"
3708      511746               2-13/16"
3709      511748               2-15/16"
3710       50938                  7/16"
3711      511750                3-1/16"
3712      511751                3-3/16"
3714      511933                 2-7/8"
3715      511935                 3-1/8"
3716      511937                 3-3/8"
3717       50940                 11/16"
3718      511939                 3-5/8"
3719      511940                 3-3/4"
3720      511941                 3-7/8"
3721      511943                 4-1/8"
3722      511944                 4-1/4"
3723      511945                 4-3/8"
3727       56406                  5/16"
3728       56408                 13/32"
3729       56410                   1/2"
3730       56412                   5/8"
3731       50955                 11/16"
3732       50951                  7/16"
3733       50113                   3/4"
3734      501064                1-1/16"
3735       50933                  9/16"
3736      501063                 15/16"
3737      501065                 1-1/8"
3738       50935                 11/16"
3739       50931                  7/16"
3740       50937                 13/16"
3741       50921                   3/4"
3742       50936                   3/4"
3743       50011                  7/16"
3744       50934                   5/8"
3745 

In [9]:
print(len(df[(df['ld'].astype(str)=='[]')&(df['name'].astype(str)=='[]')]))
df[(df['ld'].astype(str)=='[]')&(df['name'].astype(str)=='[]')][0:500]

4398


buckets external_id  \
0    Compressors Accessories & Replacement Parts       40511   
1    Compressors Accessories & Replacement Parts       40490   
2    Compressors Accessories & Replacement Parts       40492   
3    Compressors Accessories & Replacement Parts       40491   
4    Compressors Accessories & Replacement Parts       40493   
5    Compressors Accessories & Replacement Parts       40494   
6    Compressors Accessories & Replacement Parts       40495   
7    Compressors Accessories & Replacement Parts       40504   
8    Compressors Accessories & Replacement Parts      404951   
9    Compressors Accessories & Replacement Parts       40497   
10   Compressors Accessories & Replacement Parts      404952   
11   Compressors Accessories & Replacement Parts      404971   
12   Compressors Accessories & Replacement Parts     40497EZ   
13   Compressors Accessories & Replacement Parts       40498   
14   Compressors Accessories & Replacement Parts       40500   
15   Compressors Accessories & Replacement Parts     40500EZ   
16   Compressors Accessories & Replacement Parts       40502   
17   Compressors Accessories & Replacement Parts      405061   
18   Compressors Accessories & Replacement Parts      405142   
19   Compressors Accessories & Replacement Parts      405051   
20   Compressors Accessories & Replacement Parts       40503   
21   Compressors Accessories & Replacement Parts       40508   
22   Compressors Accessories & Replacement Parts       40509   
23   Compressors Accessories & Replacement Parts       40510   
24   Compressors Accessories & Replacement Parts       40514   
25   Compressors Accessories & Replacement Parts      405143   
26   Compressors Accessories & Replacement Parts       40501   
27   Compressors Accessories & Replacement Parts       40515   
28   Compressors Accessories & Replacement Parts      405151   
29   Compressors Accessories & Replacement Parts      405154   
30   Compressors Accessories & Replacement Parts      405153   
31   Compressors Accessories & Replacement Parts      405152   
32   Compressors Accessories & Replacement Parts      405155   
33   Compressors Accessories & Replacement Parts       40512   
34   Compressors Accessories & Replacement Parts       40513   
35   Compressors Accessories & Replacement Parts       40505   
36   Compressors Accessories & Replacement Parts       40545   
37   Compressors Accessories & Replacement Parts      405621   
38   Compressors Accessories & Replacement Parts      405622   
39   Compressors Accessories & Replacement Parts       40567   
42   Compressors Accessories & Replacement Parts       40568   
43   Compressors Accessories & Replacement Parts      405761   
44   Compressors Accessories & Replacement Parts      405771   
45   Compressors Accessories & Replacement Parts      405772   
46   Compressors Accessories & Replacement Parts      405741   
47   Compressors Accessories & Replacement Parts       40578   
48   Compressors Accessories & Replacement Parts      405752   
49   Compressors Accessories & Replacement Parts       40575   
50   Compressors Accessories & Replacement Parts      405781   
51   Compressors Accessories & Replacement Parts      405783   
52   Compressors Accessories & Replacement Parts      405784   
53   Compressors Accessories & Replacement Parts      405785   
54   Compressors Accessories & Replacement Parts      405786   
55   Compressors Accessories & Replacement Parts      405787   
56   Compressors Accessories & Replacement Parts      405782   
57   Compressors Accessories & Replacement Parts       40579   
58   Compressors Accessories & Replacement Parts       40577   
59   Compressors Accessories & Replacement Parts      405759   
60   Compressors Accessories & Replacement Parts       40576   
61   Compressors Accessories & Replacement Parts      405911   
67   Compressors Accessories & Replacement Parts      521260   
68   Compressors Accessories & Replacement Parts       52302   
69   Comp

In [38]:
# df[(df['name'].astype(str)!='[]')&(df['matches'].astype(str)!='[]')][0:500]

In [10]:
two=df[(df['matches'].astype(str)=='[]')]
print(len(two[two['name'].astype(str)!='[]']))
two=two[two['name'].astype(str)!='[]']
two[curation_col]=two['name'].apply(lambda x:re.sub(r'(?i)P\.?S\.?I\.?','',str(x))).apply(lambda x: re.sub(r'.+\-\s?',"['",str(x))).apply(lambda x: re.sub(r'(?<!\,)000',',000',str(x))).apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"\s?'\]",'"]',str(x))).apply(lambda x: re.sub(r"\s?\s?\s?'\s?\,\s?'",'","',str(x))).apply(lambda x: re.sub(r'200"\,"125','125","200',str(x))).apply(lambda x: re.sub(r'\["1\,500"\,"100"\,"2,000 "\]','["100","1,500","2,000"]',str(x))).apply(lambda x: re.sub(r'\s?"\]','"]',str(x)))                                                    
two[curation_col].explode().value_counts()
matchtwo=two[['external_id',curation_col]]

59


In [24]:
# two=df[(df['matches'].astype(str)=='[]')]
# # two['name']=two['name'].apply(lambda x: re.sub(r'(?i)\(|x','',str(x))).apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: re.sub(r'\s?\,\s?',',',str(x))).apply(lambda x: re.sub(r'"\]','""]',str(x))).apply(lambda x:re.sub(r'\[""|\["','["',str(x)))                                                                  
# pat=r'(?i)((?i).{0,20}dia.{0,20})|()'
# two['name']=two['product_name'].apply(lambda x: re_extract(pat, str(x)))
# two['long']=two['long_desc'].apply(lambda x: re_extract(pat, str(x)))
# two[(two['name'].astype(str)!='[]')|(two['long'].astype(str)!='[]')]

# N/A

In [11]:
na=df[(df['name'].astype(str)=='[]')&(df['matches'].astype(str)=='[]')]
print(len(na))
na[curation_col]='n/a'
matchna=na[['external_id',curation_col]]
# print(len(na)+len(matchcustom)+len(matchtwo))

3845


In [12]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchcustom))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchna))
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchtwo))

# def get_df_name(df):
#     name =[x for x in gglobals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchfinal))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchex))

matchcustom
matchna


In [11]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [13]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchcustom) 

In [14]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchna) 

In [47]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchtwo) 